In [ ]:
import tensorflow as tf
import math
import numpy as np
from nltk.tokenize import RegexpTokenizer
import pandas as pd
import itertools
from nltk.tokenize import RegexpTokenizer
from functools import partial

In [ ]:
# bible
def sparse(x):
    t = [0] * vocab_size
    t[x] = 1
    return t
def process(x):
    t = [sparse(w_i[i]) for i in x.split()]
    return t

pairs = np.load('pairs.npy')
w_i = np.load('word2index.npy').item()
i_w = np.load('index2word.npy')
vocab_size = len(i_w)
pairs.shape

In [ ]:
# conv.txt
tokenizer = RegexpTokenizer(r'\w+')
def process_sentence(x, eos=False):
#     x = x.split()
    x = tokenizer.tokenize(x)
    x = [i.lower() for i in x]
    if eos:
        x = ['eos'] + x + ['eos']
    return x

def one_hot(x):
    t = [0] * vocab_size
    t[x] = 1
    return t
    
def sent_encode(x):
    x = [w_i[i] for i in x]
    x = map(one_hot, x)
    return list(x)

data=pd.read_csv('conv.txt', delimiter='\t', error_bad_lines=False, header=None)
data.dropna(inplace=True)
inp = list(map(partial(process_sentence), data[0].values))
out = list(map(partial(process_sentence, eos=True), data[1].values))
words=np.r_[inp, out]
words = list(set(itertools.chain(*words)))
vocab_size=len(words)
i_w = dict(enumerate(words))
w_i = {v:k for k, v in i_w.items()}
inp = list(map(sent_encode, inp))
out = list(map(sent_encode, out))
print(vocab_size)

In [ ]:
input_size = vocab_size
print(input_size)
dims = [300, 200, 300]
num_layers = 3

In [ ]:
def recurrence(prev, inp):
    i = tf.reshape(inp, shape=[1, -1])
    p = tf.reshape(prev, shape=[1, -1])
    z = tf.nn.sigmoid(tf.matmul(i, enc_layers[j]['Wxz']) + tf.matmul(p, enc_layers[j]['Whz']))
    r = tf.nn.sigmoid(tf.matmul(i, enc_layers[j]['Wxr']) + tf.matmul(p, enc_layers[j]['Whr']))
    h_ = tf.nn.tanh(tf.matmul(i, enc_layers[j]['Wxh']) +
                    tf.matmul(tf.mul(p, r), enc_layers[j]['Whh']))
    h = tf.mul(tf.sub(tf.ones_like(z), z), h_) + tf.mul(z, p)
    return tf.reshape(h, [dims[j-1]])

def recurrence2(prev, inp):
    i = tf.reshape(inp, shape=[1, -1])
    p = tf.reshape(prev, shape=[1, -1])
    z = tf.nn.sigmoid(tf.matmul(i, dec_layers[j]['Wxz']) + tf.matmul(p, dec_layers[j]['Whz']))
    r = tf.nn.sigmoid(tf.matmul(i, dec_layers[j]['Wxr']) + tf.matmul(p, dec_layers[j]['Whr']))
    h_ = tf.nn.tanh(tf.matmul(i, dec_layers[j]['Wxh']) +
                    tf.matmul(tf.mul(p, r), dec_layers[j]['Whh']))
    h = tf.mul(tf.sub(tf.ones_like(z), z), h_) + tf.mul(z, p)
    return tf.reshape(h, [dims[j-1]])

In [ ]:
tf.reset_default_graph()
with tf.device('/cpu:0'):
    with tf.name_scope('encoder'):
        enc_layers = []
        incoming = tf.placeholder(shape=[None, input_size], dtype=tf.float32, name='encoder_incoming')
        enc_layers.append({'states': incoming, 'dim': input_size})
        
        for j in range(1, num_layers + 1):
            shape = [enc_layers[-1]['dim'], dims[j-1]]
            layer = dict()
            layer['Wxh'] = tf.Variable(tf.random_normal(
                shape,
                stddev=1.0 / math.sqrt(shape[0])), dtype=tf.float32)
            layer['Whh'] = tf.Variable(tf.random_normal(
                [dims[j-1], dims[j-1]], stddev=1.0 / math.sqrt(dims[j-1])), dtype=tf.float32)
            
        # layer associated with update gate
            layer['Wxz'] = tf.Variable(tf.random_normal(shape,
                stddev=1.0 / math.sqrt(shape[0])), dtype=tf.float32)
            layer['Whz'] = tf.Variable(tf.random_normal(shape=[
                dims[j-1], dims[j-1]], stddev=1.0 / math.sqrt(dims[j-1])), dtype=tf.float32)
        # layer associated with the reset gate
            layer['Wxr'] = tf.Variable(tf.random_normal(shape,
                stddev=1.0 / math.sqrt(shape[0])), dtype=tf.float32)
            layer['Whr'] = tf.Variable(tf.random_normal(shape=[
                dims[j-1], dims[j-1]],
                stddev=1.0 / math.sqrt(dims[j-1])), dtype=tf.float32)
            layer['dim'] = dims[j-1]
            layer['initial'] = tf.placeholder(shape=[dims[j-1]], dtype=tf.float32, name='initial'+str(j))
            enc_layers.append(layer)
            layer['states'] = tf.scan(recurrence, enc_layers[-2]['states'], initializer=layer['initial'], name='encoder_states'+str(j))
            layer['last'] = tf.squeeze(tf.expand_dims(tf.nn.embedding_lookup(layer['states'], 
                                                                             tf.shape(layer['states'])[0] - 1), dim=0),
                                      name='last'+str(j))

In [ ]:
with tf.device('/cpu:0'):
    with tf.name_scope('decoder'):
        
        dec_layers = []
        decoder_input = tf.placeholder(shape=[None, input_size], dtype=tf.float32, name='decoeder_input')
        dec_layers.append({'states': decoder_input, 'dim': input_size})
        
        for j in range(1, num_layers + 1):
            shape = [dec_layers[-1]['dim'], dims[j-1]]
            layer = dict()
            layer['Wxh'] = tf.Variable(tf.random_normal(
                shape,
                stddev=1.0 / math.sqrt(shape[0])), dtype=tf.float32)
            layer['Whh'] = tf.Variable(tf.random_normal(
                [dims[j-1], dims[j-1]], stddev=1.0 / math.sqrt(dims[j-1])), dtype=tf.float32)
            
        # layer associated with update gate
            layer['Wxz'] = tf.Variable(tf.random_normal(shape,
                stddev=1.0 / math.sqrt(shape[0])), dtype=tf.float32)
            layer['Whz'] = tf.Variable(tf.random_normal(shape=[
                dims[j-1], dims[j-1]], stddev=1.0 / math.sqrt(dims[j-1])), dtype=tf.float32)
        # layer associated with the reset gate
            layer['Wxr'] = tf.Variable(tf.random_normal(shape,
                stddev=1.0 / math.sqrt(shape[0])), dtype=tf.float32)
            layer['Whr'] = tf.Variable(tf.random_normal(shape=[
                dims[j-1], dims[j-1]],
                stddev=1.0 / math.sqrt(dims[j-1])), dtype=tf.float32)
            layer['dim'] = dims[j-1]
            dec_layers.append(layer)
            
            layer['states'] = tf.scan(recurrence2, dec_layers[-2]['states'], initializer=enc_layers[j]['last'], 
                                      name='decoder_states'+str(j))
            

        # softmax layer
        Why = tf.Variable(tf.random_normal(
            [dims[-1], input_size],
            stddev=1.0 / math.sqrt(dims[-1])), dtype=tf.float32)
        by = tf.Variable(tf.zeros(input_size), dtype=tf.float32)
        outputs = tf.matmul(dec_layers[-1]['states'], Why) + by
        
        # decoding step (with 'teacher forcing')

        decoder_target = tf.placeholder(shape=[None, input_size], dtype=tf.float32, name='decoder_target')
        
        loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(outputs, decoder_target))
        optimizer_op = tf.train.AdagradOptimizer(0.1).minimize(loss)

In [ ]:
saver = tf.train.Saver()
sess =  tf.Session()
saver.restore(sess, 'model.ckpt')
# sess.run(tf.initialize_all_variables())
ix = 0
avg_loss = 0

In [ ]:
# beam search output generator (explained here: http://goo.gl/g3jjYs)

def beam_generate(inp, size=2):
    inp = process_sentence(inp)
    inp = sent_encode(inp)
    out = ['eos']
    seed = [   [one_hot(w_i['eos'])]   ]
    temp = []
    for j in range(10):
        for s in seed:
            feed = {k:v for k,v in zip([l['initial'] for l in enc_layers[1:]], [np.zeros(d) for d in dims])}
            feed.update({enc_layers[0]['states'] : inp, dec_layers[0]['states']: s})        
            o = sess.run(outputs, feed_dict=feed)[-1]
            r = np.argsort(o)[-size:]
            t = list(np.argmax(s, axis=1))
#             print('r', r)
#             print('t', t)
            for i in r:
#                 print('t+i', t + [i])
                temp.append(s + [one_hot(i)])
#         print(len(temp))
        temp = temp[:size]
        seed = temp
        temp = []
    seed=[np.argmax(s, axis=1) for s in seed]
    for s in seed:
        t=''
        for j in s:
            t+=' '+i_w[j]
        print(t)
        
            
def generate_out(inp):       
    inp = process_sentence(inp)
    inp = sent_encode(inp)
    out = ['eos']
    seed = [one_hot(w_i['eos'])]
    while len(seed) < 20:
        feed = {k:v for k,v in zip([l['initial'] for l in enc_layers[1:]], [np.zeros(d) for d in dims])}
        feed.update({enc_layers[0]['states'] : inp, dec_layers[0]['states']: seed})
        o = sess.run(outputs, feed_dict=feed)[-1]
        r = i_w[np.argmax(o)]
        seed.append(o)
        out.append(r)
        if r == 'eos':
            break
    print(' '.join(out[1:-1]))

In [ ]:
print_step = 1000
for i in range(2 * len(inp)):
    ix = np.random.randint(0, len(inp))
    a = inp[ix]
    b = out[ix]
    feed = {incoming: a, decoder_target: b[1:], decoder_input : b[:-1]}
    for j in enc_layers[1:]:
        feed[j['initial']] = np.zeros(j['dim'])
        
    l, _, o = sess.run([loss, optimizer_op, outputs], feed_dict=feed)
    avg_loss += l
    if i % print_step == 0 and i != 0:
        print(avg_loss / print_step)
        avg_loss = 0
        saver.save(sess, 'checkpoint.ckpt')
        generate_out('do you have girlfriend')
        
    ix += 1

In [ ]:
# saver = tf.train.Saver()
saver.save(sess, 'model.ckpt')

In [ ]:
generate_out('thanks')

In [ ]:
data

In [ ]:
[i_w[x] for x in np.argmax(inp[0], axis=1)]